<a href="https://colab.research.google.com/github/gawankarsanket/classification_networkintrusion/blob/main/Capstone_Classification_NetSecML_Inference_BaseModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction:
We already have developed a basemodel for Network Intrusion Detection.
https://colab.research.google.com/drive/1QaCaBydr4DktlOwPIdOWjU8cAqB8OsQN
This model is trained with XGB classifier with accuracy of 99% and ready for inference.

We have saved this model in S3 bucket. Our objective is to download this model from S3 and then use it to  inference

To inference, user is expected to load input csv in same S3 from there it will be extracted and prediction file is stored back to S3 bucket.

In [ ]:
import pickle
import pandas as pd
import numpy as np


In [ ]:
pip install boto3 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.5 MB/s eta 0:00:00


# Access AWS s3 to load trained model


In [ ]:
#import python SDK ie API for AWS services
import boto3


#Access your secret keys in Python via:
from google.colab import userdata

BUCKET_NAME =userdata.get('bucketName')
ACCESS_KEY= userdata.get('accessKey')
SECRET_KEY= userdata.get('secretKey')


# enter authentication credentials
s3 = boto3.resource('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key= SECRET_KEY)

#s3 is object created which is  authrosied using access keys & secret key to  access the S3 bucket in AWS

In [ ]:
# save the filename in KEY variable
KEY = 'capstoneClassification.pkl' # replace with your object key

try:
  # we are trying to download file from s3 with name `capstoneClassification.pkl` stored in varaible KEY
  # to colab dir with name `capstoneClassification.pkl`
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'capstoneClassification.pkl')

except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise

#Load the trained model along with preporcessing objects (label enoder and selected feature list

In [ ]:
with open('capstoneClassification.pkl', 'rb') as f:
    trained_model = pickle.load(f)
    le = pickle.load(f)
    feature_selected=pickle.load(f)

In [ ]:
trained_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
le

LabelEncoder()

In [ ]:
feature_selected

array(['src_bytes', 'dst_bytes', 'hot', 'num_compromised', 'count',
       'srv_count', 'dst_host_count', 'dst_host_srv_count',
       'dst_host_same_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_serror_rate', 'dst_host_srv_serror_rate',
       'numprotocol_type', 'numservice', 'numflag'], dtype=object)

#Input file for testing

In [ ]:
input_file=pd.read_csv('/content/userinput.csv')
input_file.drop('class',axis=1,inplace=True)


In [ ]:
input_df=input_file.copy()
input_df.head()

,Unnamed: 0,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,4045743,0,icmp,ecr_i,SF,520,0,0,0,0,...,255,255,1.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0
1,2184221,0,icmp,ecr_i,SF,1032,0,0,0,0,...,255,255,1.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0
2,236495,0,tcp,http,SF,303,2406,0,0,0,...,104,255,1.0,0.0,0.01,0.05,0.0,0.0,0.0,0.0
3,1313580,0,icmp,ecr_i,SF,1032,0,0,0,0,...,255,255,1.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0
4,13623,0,tcp,http,SF,251,1835,0,0,0,...,58,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0


# Preprocess the input file

In [ ]:
def encoding(df):
  for col in df.columns:
    if df[col].dtypes=='object':
      name='num'+col                       #creating heading for that column
      df[name]=le.fit_transform(df[col])   # Column with encoded lable is added
      df.drop(col,axis=1,inplace=True)    #  column with object type will be removed

  return df

In [ ]:
input_df=encoding(input_df)

In [ ]:
input_df_selectedCols=input_df[feature_selected]
input_df_selectedCols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48984 entries, 0 to 48983
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   src_bytes                    48984 non-null  int64  
 1   dst_bytes                    48984 non-null  int64  
 2   hot                          48984 non-null  int64  
 3   num_compromised              48984 non-null  int64  
 4   count                        48984 non-null  int64  
 5   srv_count                    48984 non-null  int64  
 6   dst_host_count               48984 non-null  int64  
 7   dst_host_srv_count           48984 non-null  int64  
 8   dst_host_same_srv_rate       48984 non-null  float64
 9   dst_host_same_src_port_rate  48984 non-null  float64
 10  dst_host_serror_rate         48984 non-null  float64
 11  dst_host_srv_serror_rate     48984 non-null  float64
 12  numprotocol_type             48984 non-null  int64  
 13  numservice      

In [ ]:
input_df_selectedCols.head()

,src_bytes,dst_bytes,hot,num_compromised,count,srv_count,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_same_src_port_rate,dst_host_serror_rate,dst_host_srv_serror_rate,numprotocol_type,numservice,numflag
0,520,0,0,0,481,481,255,255,1.0,1.00,0.0,0.0,0,14,5
1,1032,0,0,0,511,511,255,255,1.0,1.00,0.0,0.0,0,14,5
2,303,2406,0,0,5,5,104,255,1.0,0.01,0.0,0.0,1,22,5
3,1032,0,0,0,511,511,255,255,1.0,1.00,0.0,0.0,0,14,5
4,251,1835,0,0,3,3,58,255,1.0,0.02,0.0,0.0,1,22,5


# PREDICTION

In [ ]:
def prediction_function(xtest):
  ypred=trained_model.predict(xtest)
  return ypred

In [ ]:
ypred=prediction_function(input_df_selectedCols)
ypred

array([1, 1, 0, ..., 1, 0, 1])

#Map the output back to Input file

In [ ]:
input_file['IsMalicious']=ypred
input_file.head(3)

,Unnamed: 0,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,IsMalicious
0,4045743,0,icmp,ecr_i,SF,520,0,0,0,0,...,255,1.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0,1
1,2184221,0,icmp,ecr_i,SF,1032,0,0,0,0,...,255,1.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0,1
2,236495,0,tcp,http,SF,303,2406,0,0,0,...,255,1.0,0.0,0.01,0.05,0.0,0.0,0.0,0.0,0


In [ ]:
output_mappping={0:'Normal',1:'Attack'}
input_file['class']=input_file['IsMalicious'].map(output_mappping)

input_file.head()

,Unnamed: 0,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,IsMalicious,class
0,4045743,0,icmp,ecr_i,SF,520,0,0,0,0,...,1.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0,1,Attack
1,2184221,0,icmp,ecr_i,SF,1032,0,0,0,0,...,1.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0,1,Attack
2,236495,0,tcp,http,SF,303,2406,0,0,0,...,1.0,0.0,0.01,0.05,0.0,0.0,0.0,0.0,0,Normal
3,1313580,0,icmp,ecr_i,SF,1032,0,0,0,0,...,1.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0,1,Attack
4,13623,0,tcp,http,SF,251,1835,0,0,0,...,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0,0,Normal


#Push This file to S3 as predicted output file

In [ ]:
#Saving the output file

input_file.to_csv('Prediction.csv', index=False)

In [ ]:
#Creating S3 object that will  push the file
push_to_s3= boto3.client('s3',aws_access_key_id=ACCESS_KEY,aws_secret_access_key=SECRET_KEY)

In [ ]:
#Using object pushing/ uploading file to S3
push_to_s3.upload_file('/content/Prediction.csv',BUCKET_NAME, 'Prediction.csv')